In [1]:
import sys
import os
import pandas as pd

# Caminho até a raiz do projeto (a pasta que contém "titulospub/")
caminho_raiz = os.path.abspath("Z:\\Chila\\projetos\\calculadora_titulos_publicos")
if caminho_raiz not in sys.path:
    sys.path.append(caminho_raiz)


from titulospub import *
from titulospub.core.auxilio import codigo_vencimento_bmf

In [13]:
import pandas as pd

from titulospub.dados.orquestrador import VariaveisMercado
from titulospub.utils import adicionar_dias_uteis
from titulospub.core.ntnf.calculo_ntnf import calcular_ntnf
from titulospub.core.auxilio import vencimento_codigo_bmf
from titulospub.core.di.calculo_di import calculo_dv01_di


class NTNF_T:
    """
    Classe para cálculo e gestão de títulos NTN-F (Nota do Tesouro Nacional - Série F).
    
    Esta classe encapsula todos os cálculos relacionados aos títulos NTN-F,
    incluindo preços, DV01, carregamento e hedge DI.
    """
    
    def __init__(self, 
                 data_vencimento_titulo: str, 
                 data_base: str = None, 
                 dias_liquidacao: int = 1,
                 taxa: float = None,
                 premio: float = None,
                 di: float = None,
                 quantidade: float = 50000, 
                 cdi: float = None,  
                 feriados: list = None,
                 variaveis_mercado: VariaveisMercado = None):
        """
        Inicializa uma instância do título NTN-F.
        
        Args:
            data_vencimento_titulo: Data de vencimento do título
            data_base: Data base para cálculos (default: hoje)
            dias_liquidacao: Dias para liquidação (default: 1)
            taxa: Taxa de juros do título
            premio: Prêmio sobre DI
            di: Taxa DI de referência
            quantidade: Quantidade de títulos
            cdi: Taxa CDI
            feriados: Lista de feriados
            variaveis_mercado: Instância de VariaveisMercado
        """
        # Configuração inicial
        self._vm = variaveis_mercado or VariaveisMercado()
        self._feriados = feriados if feriados is not None else self._vm.get_feriados()
        self._cdi = cdi if cdi is not None else self._vm.get_cdi()
        
        # Parâmetros de entrada
        self._taxa = float(taxa) if taxa is not None else None
        self._premio = float(premio) if premio is not None else None
        self._di = float(di) if di is not None else None
        self._quantidade = float(quantidade)
        
        # Configuração de datas
        self._configurar_datas(data_vencimento_titulo, data_base, dias_liquidacao)
        
        # Configuração do título
        self._configurar_titulo()
        
        # Configuração da taxa
        self._configurar_taxa()
        
        # Configuração DI
        self._configurar_di()
        
        # Inicialização de atributos derivados
        self._inicializar_atributos_derivados()
        
        # Cálculos iniciais
        self._calcular()
        self._hedge_di = self._calcular_hedge_di()
        self._financeiro = self._quantidade * self._pu_d0

    # ==================== CONFIGURAÇÃO PRIVADA ====================
    
    def _configurar_datas(self, data_vencimento_titulo: str, data_base: str, dias_liquidacao: int):
        """Configura as datas do título."""
        self._dias_liquidacao = dias_liquidacao
        self._data_vencimento_titulo = pd.to_datetime(data_vencimento_titulo)
        self._data_base = (pd.to_datetime(data_base).normalize() 
                          if data_base 
                          else pd.Timestamp.today().normalize())
        self._data_liquidacao = adicionar_dias_uteis(
            data=self._data_base,
            n_dias=dias_liquidacao,
            feriados=self._feriados
        )
    
    def _configurar_titulo(self):
        """Configura informações básicas do título."""
        self._nome = f"NTNF {self._data_vencimento_titulo.month}/{self._data_vencimento_titulo.year}"
        
        # Busca taxa ANBIMA
        df_ntnf = self._vm.get_anbimas()["NTN-F"]
        linha = df_ntnf[df_ntnf["VENCIMENTO"] == self._data_vencimento_titulo]
        
        if linha.empty:
            raise ValueError(f"Vencimento {self._data_vencimento_titulo.date()} não encontrado na ANBIMA.")
        
        self._anbima = linha.squeeze()["ANBIMA"]
    
    def _configurar_taxa(self):
        """Configura a taxa do título baseada nos parâmetros fornecidos."""
        if self._taxa is None:
            if (self._premio is None) or (self._di is None):
                self._taxa = float(self._anbima)
            else:
                self._taxa = float(self._di + self._premio / 100)
        else:
            self._taxa = float(self._taxa)
    
    def _configurar_di(self):
        """Configura parâmetros relacionados ao DI."""
        self._di_ref = vencimento_codigo_bmf(
            data_vencimento=self._data_vencimento_titulo,
            prefixo="DI1"
        )
        curva_di = self._vm.get_bmf()["DI"]
        self._ajuste_di = curva_di.loc[curva_di["DI"] == self._di_ref].squeeze()["ADJ"]
        self._premio_anbima = (self._anbima - self._ajuste_di) * 100
    
    def _inicializar_atributos_derivados(self):
        """Inicializa atributos que serão calculados posteriormente."""
        self._pu_d0 = None
        self._pu_termo = None
        self._pu_carregado = None
        self._dv01 = None
        self._carrego_brl = None
        self._carrego_bps = None
        self._hedge_di = None
        self._financeiro = None

    # ==================== PROPRIEDADES DE ENTRADA ====================
    
    @property
    def taxa(self):
        """Taxa de juros do título."""
        return self._taxa
    
    @taxa.setter
    def taxa(self, v):
        self._taxa = float(v)
        self._calcular()
        self._atualizar_hedge_e_financeiro()
    
    @property
    def premio(self):
        """Prêmio sobre DI."""
        return self._premio
    
    @premio.setter
    def premio(self, v):
        self._premio = float(v) if v is not None else None
        self._atualizar_taxa_premio_di()
        self._calcular()
        self._atualizar_hedge_e_financeiro()

    @property
    def di(self):
        """Taxa DI de referência."""
        return self._di
    
    @di.setter
    def di(self, v):
        self._di = float(v) if v is not None else None
        self._atualizar_taxa_premio_di()
        self._calcular()
        self._atualizar_hedge_e_financeiro()
    
    @property
    def data_base(self):
        """Data base para cálculos."""
        return self._data_base
    
    @data_base.setter
    def data_base(self, v):
        self._data_base = pd.to_datetime(v).normalize()
        self._atualizar_data_liquidacao()
        self._calcular()
        self._atualizar_hedge_e_financeiro()
    
    @property
    def data_liquidacao(self):
        """Data de liquidação do título."""
        return self._data_liquidacao
    
    @data_liquidacao.setter
    def data_liquidacao(self, v):
        self._data_liquidacao = pd.to_datetime(v).normalize()
        self._calcular()
        self._atualizar_hedge_e_financeiro()
    
    @property
    def quantidade(self):
        """Quantidade de títulos."""
        return self._quantidade
    
    @quantidade.setter
    def quantidade(self, v):
        if v <= 0:
            raise ValueError("Quantidade deve ser maior que zero")
        
        self._ajustar_valores_para_quantidade(v)
        self._hedge_di = self._calcular_hedge_di()

    @property
    def dias_liquidacao(self) -> int:
        """Dias para liquidação."""
        return self._dias_liquidacao
    
    @dias_liquidacao.setter
    def dias_liquidacao(self, n: int):
        self._dias_liquidacao = int(n)
        self._atualizar_data_liquidacao()
        self._calcular()
        self._atualizar_hedge_e_financeiro()

    @property
    def financeiro(self):
        """Valor financeiro total."""
        return self._financeiro

    @financeiro.setter
    def financeiro(self, v):
        if v <= 0:
            raise ValueError("Financeiro deve ser maior que zero")
            
        if self._pu_d0 == 0:
            raise ValueError("PU_D0 não pode ser zero para calcular quantidade")
        
        self._ajustar_valores_para_financeiro(v)
        self._hedge_di = self._calcular_hedge_di()

    # ==================== PROPRIEDADES SOMENTE LEITURA ====================
    
    @property
    def pu_d0(self):
        """Preço unitário à vista."""
        return self._pu_d0
    
    @property
    def pu_termo(self):
        """Preço unitário a termo."""
        return self._pu_termo
    
    @property
    def pu_carregado(self):
        """Preço unitário carregado."""
        return self._pu_carregado
    
    @property
    def dv01(self):
        """DV01 do título."""
        return self._dv01
    
    @property
    def carrego_brl(self):
        """Carregamento em BRL."""
        return self._carrego_brl
    
    @property
    def carrego_bps(self):
        """Carregamento em pontos base."""
        return self._carrego_bps
    
    @property
    def hedge_di(self):
        """Hedge DI calculado."""
        return self._hedge_di

    # ==================== MÉTODOS DE CÁLCULO ====================
    
    def _calcular(self):
        """Método principal de cálculo do título."""
        res = calcular_ntnf(
            data=self._data_base,
            data_liquidacao=self._data_liquidacao,
            data_vencimento=self._data_vencimento_titulo,
            taxa=self._taxa,
            cdi=self._cdi,
            feriados=self._feriados
        )
        
        # Armazena resultados
        self._pu_d0 = res["pu_d0"]
        self._pu_termo = res["pu_termo"]
        self._pu_carregado = res["pu_carregado"]
        self._dv01 = res["dv01"] * self._quantidade
        self._carrego_brl = res["carrego_brl"] * self._quantidade
        self._carrego_bps = res["carrego_bps"]
        
        # Atualiza financeiro
        self._financeiro = self._quantidade * self._pu_d0
    
    def _calcular_hedge_di(self):
        """Calcula o hedge DI para o título."""
        dv_di = calculo_dv01_di(taxa=self._ajuste_di, codigo=self._di_ref)
        return int(self._dv01 / dv_di)
    
    def _atualizar_taxa_premio_di(self):
        """Atualiza a taxa baseada em prêmio e DI quando ambos estão definidos."""
        if self._premio is not None and self._di is not None:
            self._taxa = float(self._di + self._premio / 100)
    
    def _atualizar_data_liquidacao(self):
        """Atualiza a data de liquidação baseada nos dias de liquidação."""
        self._data_liquidacao = adicionar_dias_uteis(
            data=self._data_base,
            n_dias=self._dias_liquidacao,
            feriados=self._feriados
        )
    
    def _atualizar_hedge_e_financeiro(self):
        """Atualiza hedge DI e financeiro após mudanças."""
        self._hedge_di = self._calcular_hedge_di()
        self._financeiro = self._quantidade * self._pu_d0
    
    def _ajustar_valores_para_quantidade(self, nova_quantidade):
        """Ajusta valores quando a quantidade é alterada."""
        quantidade_anterior = getattr(self, "_quantidade", 1)
        
        # Normaliza valores para unidade
        self._dv01 = self._dv01 / quantidade_anterior
        self._carrego_brl = self._carrego_brl / quantidade_anterior
        
        # Atualiza quantidade
        self._quantidade = float(nova_quantidade)
        
        # Reaplica multiplicação
        self._dv01 *= self._quantidade
        self._carrego_brl *= self._quantidade
        self._financeiro = self._quantidade * self._pu_d0
    
    def _ajustar_valores_para_financeiro(self, novo_financeiro):
        """Ajusta valores quando o financeiro é alterado."""
        quantidade_anterior = getattr(self, "_quantidade", 1)
        
        # Normaliza valores para unidade
        self._dv01 = self._dv01 / quantidade_anterior
        self._carrego_brl = self._carrego_brl / quantidade_anterior
        
        # Calcula nova quantidade
        self._financeiro = float(novo_financeiro)
        self._quantidade = round(self._financeiro / self._pu_d0, 6)
        
        # Reaplica multiplicação
        self._dv01 *= self._quantidade
        self._carrego_brl *= self._quantidade


In [2]:
faca = NTNF(data_vencimento_titulo="2031-01-01")

✅ Usando cache existente de ANBIMAS completo.
✅ Usando cache existente de BMF completo.


In [3]:
faca.hedge_di

667

In [ ]:
# Teste da classe organizada
faca = NTNF_T(data_vencimento_titulo="2031-01-01", quantidade=1)

print(f"Taxa: {faca.taxa:.4f}")
print(f"PU D0: {faca.pu_d0:.6f}")
print(f"DV01: {faca.dv01:.2f}")
print(f"Hedge DI: {faca.hedge_di}")
print(f"Financeiro: {faca.financeiro:.2f}")


In [2]:
vm = VariaveisMercado()
vm.atualizar_tudo()

🔄 Atualizando variáveis de mercado...
📡 Buscando feriados via scraping...
📡 Calculando IPCA dict...
📡 Buscando CDI...
📡 Realizando scraping ANBIMA...
♻️ Cache salvo para todos os títulos ANBIMA.
📡 Realizando scraping BMF...
♻️ Cache salvo para todos os contrados de DI e DAP.
📡 Realizando scraping VNA_LFT...
♻️ Cache salvo para VNA_LFT.
✅ Atualização concluída.


In [ ]:
import pandas as pd

from titulospub.dados.orquestrador import VariaveisMercado
from titulospub.utils import adicionar_dias_uteis
from titulospub.core.ntnf.calculo_ntnf import calcular_ntnf
from titulospub.core.auxilio import vencimento_codigo_bmf
from titulospub.core.di.calculo_di import calculo_dv01_di

class NTNF:
    def __init__(self, data_vencimento_titulo: str, 
                       data_base: str=None, 
                       dias_liquidacao: int=1,
                       taxa: float=None,
                       premio: float=None,
                       di: float=None,
                       quantidade=50000, 
                       cdi: float=None,  
                       feriados: list=None,
                       variaveis_mercado: VariaveisMercado | None = None):

        # Injete uma instância para evitar recriar VariaveisMercado várias vezes
        self._vm = variaveis_mercado or VariaveisMercado()

        # Variáveis globais
        self._feriados   = feriados   if feriados   is not None else self._vm.get_feriados()
        self._cdi        = cdi        if cdi        is not None else self._vm.get_cdi()

        self._taxa = float(di) if taxa is not None else None
        self._premio = float(premio) if premio is not None else None
        self._di = float(di) if di is not None else None

        # Datas
        self._dias_liquidacao = dias_liquidacao
        self._data_vencimento_titulo = pd.to_datetime(data_vencimento_titulo)
        self._data_base = pd.to_datetime(data_base).normalize() if data_base else pd.Timestamp.today().normalize()
        self._data_liquidacao =  adicionar_dias_uteis(data=self._data_base,
                                                           n_dias=dias_liquidacao,
                                                           feriados=self._feriados)
        
        # Quantidade de titulos
        self._quantidade = quantidade
        self._financeiro = None  # Será calculado após _calcular()
        
        # Nome
        self._nome = f"NTNF {self._data_vencimento_titulo.month}/{self._data_vencimento_titulo.year}"

        # Taxa default pela ANBIMA do vencimento
        df_ntnf = self._vm.get_anbimas()["NTN-F"]
        linha = df_ntnf[df_ntnf["VENCIMENTO"] == self._data_vencimento_titulo]
        if linha.empty:
            raise ValueError(f"Vencimento {self._data_vencimento_titulo.date()} não encontrado na ANBIMA.")
        self._anbima = linha.squeeze()["ANBIMA"]

        if self._taxa is None:
            if (self._premio is None) or (self._di is None):
                self._taxa =  float(self._anbima)
            else:
                self._taxa = float(self._di + self._premio / 100)
        else:
            self._taxa = float(taxa)


        #self._taxa = float(taxa) if taxa is not None else float(self._anbima)

        # --- DI ---
        self._di_ref = vencimento_codigo_bmf(data_vencimento=self._data_vencimento_titulo,
                                             prefixo="DI1")
        curva_di = self._vm.get_bmf()["DI"]

        self._ajuste_di = float(curva_di.loc[curva_di["DI"] == self._di_ref]["ADJ"])
        

        self._premio_anbima = (self._anbima - self._ajuste_di) * 100

        # Atributos DERIVADOS (serão preenchidos em _calcular)
        self._pu_d0 = None
        self._pu_termo = None
        self._pu_carregado = None
        self._dv01 = None
        self._carrego_brl = None
        self._carrego_bps = None
        self._hedge_di = None

        # Calcula já na criação
        self._calcular()
        self._hedge_di = self._calcular_hedge_di()
        # Calcula o financeiro após ter o pu_d0
        self._financeiro = self._quantidade * self._pu_d0


    @property
    def taxa(self): return self._taxa
    @taxa.setter
    def taxa(self, v):
        self._taxa = float(v)
        self._calcular()
    
    @property
    def premio(self): 
        return self._premio

    @premio.setter
    def premio(self, v):
        self._premio = float(v) if v is not None else None
        self._atualizar_taxa_premio_di()
        self._calcular()

    @property
    def di(self): 
        return self._di

    @di.setter
    def di(self, v):
        self._di = float(v) if v is not None else None
        self._atualizar_taxa_premio_di()
        self._calcular()

    
    
    @property
    def data_base(self): return self._data_base
    @data_base.setter
    def data_base(self, v):
        self._data_base = pd.to_datetime(v).normalize()
        
        self._calcular()
    
    @property
    def data_liquidacao(self): return self._data_liquidacao
    @data_liquidacao.setter
    def data_liquidacao(self, v):
        self._data_liquidacao = pd.to_datetime(v).normalize()
    
        self._calcular()
    
    @property
    def quantidade(self):
        return self._quantidade

    @quantidade.setter
    def quantidade(self, v):
        if v <= 0:
            raise ValueError("Quantidade deve ser maior que zero")
            
        # Usa 1 como padrão para a primeira atribuição
        quantidade_anterior = getattr(self, "_quantidade", 1)

        # Ajusta valores para a unidade
        self._dv01 = self._dv01 / quantidade_anterior
        self._carrego_brl = self._carrego_brl / quantidade_anterior

        # Atualiza a quantidade
        self._quantidade = float(v)
        
        # Atualiza o financeiro baseado na nova quantidade
        self._financeiro = self._quantidade * self._pu_d0

        # Reaplica multiplicação
        self._dv01 *= self._quantidade
        self._carrego_brl *= self._quantidade
        self._hedge_di = self._calcular_hedge_di()

    
    @property
    def dias_liquidacao(self) -> int:
        return self._dias_liquidacao
    @dias_liquidacao.setter
    def dias_liquidacao(self, n: int):
        self._dias_liquidacao = int(n)
        self._data_liquidacao = adicionar_dias_uteis(
                                                     data=self._data_base,
                                                     n_dias=self._dias_liquidacao,
                                                     feriados=self._feriados
                                                    )
        self._calcular()

    # -------- Propriedade financeiro --------
    @property
    def financeiro(self):
        return self._financeiro

    @financeiro.setter
    def financeiro(self, v):
        if v <= 0:
            raise ValueError("Financeiro deve ser maior que zero")
            
        if self._pu_d0 == 0:
            raise ValueError("PU_D0 não pode ser zero para calcular quantidade")
            
        # Usa 1 como padrão para a primeira atribuição
        quantidade_anterior = getattr(self, "_quantidade", 1)

        # Ajusta valores para a unidade
        self._dv01 = self._dv01 / quantidade_anterior
        self._carrego_brl = self._carrego_brl / quantidade_anterior

        # Calcula nova quantidade baseada no financeiro
        self._financeiro = float(v)
        self._quantidade = round(self._financeiro / self._pu_d0, 6)

        # Reaplica multiplicação
        self._dv01 *= self._quantidade
        self._carrego_brl *= self._quantidade
        self._hedge_di = self._calcular_hedge_di()

    

    # -------- Método central de cálculo --------
    def _calcular_hedge_di(self):
        dv_di = calculo_dv01_di(taxa=self._ajuste_di, codigo=self._di_ref)

        return int(self._dv01 / dv_di)


    def _calcular(self):
        res = calcular_ntnf(
            data=self._data_base,
            data_liquidacao=self._data_liquidacao,
            data_vencimento=self._data_vencimento_titulo,
            taxa=self._taxa,
            cdi=self._cdi,
            feriados=self._feriados
        )
        # guarda os derivados
        self._pu_d0         = res["pu_d0"]
        self._pu_termo      = res["pu_termo"]
        self._pu_carregado  = res["pu_carregado"]
        self._dv01          = res["dv01"] * self._quantidade
        self._carrego_brl       = res["carrego_brl"]* self._quantidade
        self._carrego_bps       = res["carrego_bps"]
        
        # Atualiza o financeiro baseado na quantidade atual
        self._financeiro = self._quantidade * self._pu_d0
        
    
    def _atualizar_taxa_premio_di(self):
        """Atualiza a taxa baseada em premio e di quando ambos estão definidos"""
        if self._premio is not None and self._di is not None:
            self._taxa = float(self._di + self._premio / 100)
            self._calcular()
    
    
    

    # -------- Propriedades somente-leitura para derivados --------

    @property
    def pu_d0(self): return self._pu_d0
    @property
    def pu_termo(self): return self._pu_termo
    @property
    def pu_carregado(self): return self._pu_carregado
    @property
    def dv01(self): return self._dv01
    @property
    def carrego_brl(self): return self._carrego_brl
    @property
    def carrego_bps(self): return self._carrego_bps



In [36]:
di = "DI1F31"
vencimento = pd.to_datetime("2031-01-01")
ajuste = 13.563

eq =  int(50000 * (dv_faca) / (dv_di))

eq
 

667

In [3]:
faca = NTNF(data_vencimento_titulo="2031-01-01")

✅ Usando cache existente de ANBIMAS completo.
✅ Usando cache existente de BMF completo.


Z:\Chila\projetos\calculadora_titulos_publicos\titulospub\core\ntnf\titulo_ntnf.py:70: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._ajuste_di = float(curva_di.loc[curva_di["DI"] == self._di_ref]["ADJ"])


In [6]:
faca._hedge_di

371

In [18]:
dv_faca = faca._dv01
dv_faca

0.3153109999999515

In [15]:
dv_di = calculo_dv01_di(13.563, codigo=di)
dv_di

23.631077999998524

In [ ]:
# --- DI ---
        self._di_ref = vencimento_codigo_bmf(data_vencimento=self._data_vencimento_titulo,
                                             prefixo="DI1")
        curva_di = self._vm._bmf["DI"]

        self._ajuste_di = curva_di.loc[curva_di["DI"] == self._di_ref]

In [21]:
df = vm._bmf["DI"]
df.loc[df["DI"]=="DI1F35"]

,DATA,DATA_VENCIMENTO,DI,ADJ
35,2025-09-22,2035-01-02,DI1F35,13.5
